https://arxiv.org/pdf/2212.10496.pdf
langchain version: https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb
video course: https://youtu.be/SaDzIVkYqyY?si=RmuR3tra6QFWGWt7

In [43]:
!wget -O ./tmp/Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
file_path = "./tmp/Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

zsh:1: no matches found: https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
from rich.pretty import pprint
from typing import Any


def pretty_print(title: str = None, content: Any = None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

In [45]:
SIM_TOP_K = 5
RERANK_TOP_K = 5
WIN_SZ = 5

In [46]:
from typing import List

from llama_index.core import (ServiceContext, SimpleDirectoryReader,
                              VectorStoreIndex)
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.indices.base import BaseIndex
from llama_index.core.base.llms.base import BaseLLM
from llama_index.core.llms.utils import LLMType
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq

#llm: BaseLLM = Ollama(model="gemma:2b-instruct", temperature=0)
llm: BaseLLM = Groq(model="mixtral-8x7b-32768", temperature=0)

def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )


embs = "local:BAAI/bge-small-en-v1.5"  # OpenAIEmbedding(model=OpenAIEmbeddingModelType.TEXT_EMBED_ADA_002)
service_context: ServiceContext = create_service_context(llm, embs)

input_files: List[str] = [file_path]
docs: SimpleDirectoryReader = SimpleDirectoryReader(
    input_files=input_files,
).load_data()


vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    show_progress=True,
)

base_retriever = vector_index.as_retriever()

/var/folders/3k/3105m_0n0jq3mpst61fnchtr0000gn/T/ipykernel_16186/3375871418.py:22: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  return ServiceContext.from_defaults(
Generating embeddings: 100%|██████████| 160/160 [00:13<00:00, 11.50it/s]


In [47]:
from typing import List

from llama_index.core import PromptTemplate, QueryBundle
from llama_index.core.base.llms.base import BaseLLM
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore


class HyDERetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, hypo_gen_model: BaseLLM):
        self.base_retriever = base_retriever
        self.hypo_gen_model = hypo_gen_model
        self.hypothesis_template = PromptTemplate(
            """Write a hypothesis paper about question as you can.

            Only return the paper content without any other information, ie. leading text and so on.
            
            Question: {question}

            """
        )
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return []
    
    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query_str: str = query_bundle.query_str
        gen_answer = self.hypo_gen_model.complete(
            self.hypothesis_template.format(question=query_str)
        ).text.strip()
        pretty_print("Generated info", gen_answer)
        return await self.base_retriever.aretrieve(gen_answer)

In [48]:
from llama_index.core.indices.postprocessor import (
    MetadataReplacementPostProcessor,
    SentenceTransformerRerank,
)
from llama_index.core.base.llms.base import BaseLLM
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.ollama import Ollama
from llama_index.llms.groq import Groq
from llama_index.core.response_synthesizers.base import BaseSynthesizer
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers.type import ResponseMode
from llama_index.core.base.response.schema import RESPONSE_TYPE

query_text = """What is the "CODE OF BUSINESS CONDUCT AND ETHICS" of BAIDU Inc.? 
    """


# hypo_gen_model: BaseLLM = Ollama(model="gemma:2b-instruct", temperature=1.5)
hypo_gen_model: BaseLLM = Groq(model="mixtral-8x7b-32768", temperature=1.5)
hyde_retriever = HyDERetriever(base_retriever, hypo_gen_model)
postproc: BaseNodePostprocessor = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)
rerank: BaseNodePostprocessor = SentenceTransformerRerank(
    top_n=RERANK_TOP_K, model="BAAI/bge-reranker-base"
)
response_synthesizer: BaseSynthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode=ResponseMode.REFINED,
)
final_res: RESPONSE_TYPE = await RetrieverQueryEngine(
    hyde_retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[postproc, rerank],
).aquery(query_text)

Generated info


'In the contemporary business landscape, the significance of a robust Code of Business Conduct and Ethics cannot be overstated. This is especially true for multinational technology companies like Baidu Inc., which operate in a rapidly evolving industry that is deeply intertwined with societal values and ethical considerations. This paper aims to delve into the Code of Business Conduct and Ethics of Baidu Inc., providing an overview of the company\'s ethical framework and its commitment to responsible corporate practices.\n\nI. Introduction\n\nBaidu Inc., often referred to as the "Google of China," is a leading multinational technology company specializing in artificial intelligence (AI), internet search, and various related services. As a prominent player in the global technology landscape, Baidu is responsible for upholding a strong code of ethics that guides its decision-making processes and operations.\n\nII. Baidu Inc.\'s Code of Business Conduct and Ethics: An Overview\n\nBaidu\'s Code of Business Conduct and Ethics outlines the company\'s commitment to maintaining the highest standards of professional and ethical conduct. The code applies to all Baidu employees, officers, and directors, ensuring that the company\'s business practices align with its core values and legal requirements.\n\nIII. Key Components of Baidu\'s Code of Business Conduct and Ethics\n\n1. Compliance with Laws, Regulations, and Company Policies: Baidu is committed to adhering to all applicable laws, regulations, and company policies. This includes, but is not limited to, competition law, insider trading laws, and anti-bribery and corruption regulations.\n2. Conflicts of Interest: Baidu employees are required to disclose and manage any actual or potential conflicts of interest to ensure that their actions do not harm the company or its stakeholders.\n3. Fair Dealing: Baidu is dedicated to treating all employees, customers, and partners fairly and ethically. This includes fostering a work environment that is free from discrimination and harassment.\n4. Protection and Proper Use of Company Assets: Baidu employees are expected to protect and use the company\'s assets, including intellectual property, in a responsible and efficient manner.\n5. Confidentiality: Baidu is committed to maintaining the confidentiality of its proprietary and confidential information, as well as that of its partners and customers.\n6. Corporate Opportunities: Employees are prohibited from taking advantage of corporate opportunities for personal gain or diverting such opportunities from Baidu.\n7. Financial Integrity: Baidu is dedicated to maintaining accurate and transparent financial records, ensuring the integrity of its financial reporting.\n8. Reporting and Accountability: Employees are encouraged to report any suspected violations of the Code of Business Conduct and Ethics, and the company guarantees that all reports will be handled confidentially and without retaliation.\n\nIV. Conclusion\n\nBaidu Inc.\'s Code of Business Conduct and Ethics serves as the backbone of its corporate responsibility framework, ensuring that the company remains committed to ethical business practices and legal compliance. By fostering a culture of integrity, Baidu can maintain the trust of its stakeholders and continue to thrive in the ever-evolving technology landscape.'

In [51]:
from IPython.display import HTML
from llama_index.llms.groq import Groq
from llama_index.llms.ollama import Ollama

pretty_print("final_res.response", final_res.response)

fmt_model: BaseLLM = Groq(model="mixtral-8x7b-32768", temperature=1.5)
#Ollama(model="gemma:2b-instruct", temperature=1.5, request_timeout=120)
final_text: str = fmt_model.complete(
    f"""Format the text in HTML with appropriate markup for clarity and organization, bullet points if necessary, and so on.
    Do not add any additional information, only format the text.

    text: 
    
    {final_res.response}

    """
).text.strip()


HTML(final_text)

final_res.response


'The CODE OF BUSINESS CONDUCT AND ETHICS of Baidu Inc. is a document that applies to all employees, officers, and directors of the company. It contains general guidelines for conducting the business of the company consistent with the highest standards of business ethics.'